In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from rtseg.cells.plotting import generate_fork_plot, generate_abins_lbins, get_bulk_init_area
import numpy as np
import sys
from pathlib import Path
from skimage.io import imsave, imread
import zarr
from numcodecs import Zlib
from rtseg.utils.db_ops import read_from_db
from tifffile import imwrite # used for finer compression things
import h5py
import pandas as pd
from os.path import join
import polars as pl
%matplotlib qt5
from rtseg.utils.param_io import load_params
from rtseg.utils.disk_ops import read_files
import os
from skimage.metrics import structural_similarity as ssim
import pysal.lib
import esda.moran 
import pysal.explore
from scipy import stats
from scipy.spatial.distance import pdist, squareform 
from scipy.fft import fft2, fftfreq
from scipy.special import rel_entr

C:\Users\oscbr226\Desktop\Initiation_Lib_Tweezer\Code\realtimeAnalysis\.venv\Lib\site-packages\spaghetti\network.py:41: FutureWarning: The next major release of pysal/spaghetti (2.0.0) will drop support for all ``libpysal.cg`` geometries. This change is a first step in refactoring ``spaghetti`` that is expected to result in dramatically reduced runtimes for network instantiation and operations. Users currently requiring network and point pattern input as ``libpysal.cg`` geometries should prepare for this simply by converting to ``shapely`` geometries.
  warnings.warn(dep_msg, FutureWarning, stacklevel=1)


In [3]:
param = load_params(Path('C:/Users/oscbr226/Desktop/Initiation_Lib_Tweezer/Experiments/EXP-25-CD0119/expt_params.yaml'), ref_type='expt')
param.Save.directory = 'C:/Users/oscbr226/Desktop/Initiation_Lib_Tweezer/Experiments/EXP-25-CD0119'
save_dir = Path(param.Save.directory)

In [4]:
use_pyarrow = True
columns_to_extract = ['area', 'length', 'normalized_internal_x', 'normalization_counts', 'position', 'trap']
all_data = pl.read_parquet(save_dir / Path('forks_all.parquet'), use_pyarrow=use_pyarrow, columns=columns_to_extract)

areas = all_data['area'].to_numpy()
longs = all_data['normalized_internal_x'].to_numpy()
lengths = all_data['length'].to_numpy()
counts = all_data['normalization_counts'].to_numpy()

bin_scale = param.Forkplots.bin_scale
heatmap_threshold = param.Forkplots.heatmap_threshold
pixel_size = param.Forkplots.pixel_size

heatmap, mean_cell_lengths, abins, lbins, (x, y) = generate_fork_plot(areas, lengths, longs, counts,
                bin_scale=bin_scale,
                pixel_size=pixel_size,
                heatmap_threshold=heatmap_threshold)



In [5]:
min_length = param.Forkplots.min_length
arb_div_area = param.Forkplots.arb_div_area
init_area = param.Forkplots.init_area_cv

init_area = get_bulk_init_area(areas, counts, longs, lengths, min_length=min_length, pixel_size=pixel_size, arb_div_area=arb_div_area)


In [6]:
plt.matshow(heatmap, aspect='auto', interpolation='none', extent=[x[0], x[-1], y[-1], y[0]], origin='upper', cmap='jet')
plt.plot(-0.5 * mean_cell_lengths, y, 'w', linewidth=2)
plt.plot(0.5 * mean_cell_lengths, y, 'w', linewidth=2)
plt.axhline(init_area, color='red', linestyle='--', linewidth=2)
plt.xlim(-3, 3) 
plt.ylim(3,y[0])
plt.xlabel('Cell long axis (µm)')
plt.ylabel('Cell size (µm$^2$)')

Text(0, 0.5, 'Cell size (µm$^2$)')

In [7]:
init_area_cv = param.Forkplots.init_area_cv
init_area_low = init_area - init_area_cv*init_area
init_area_high = init_area + init_area_cv*init_area

abins_inds_around_init = np.where(np.logical_and(init_area_low <= abins, abins <= init_area_high))
abins_inds_around_init = abins_inds_around_init[0]
abins_inds_around_init = np.append(abins_inds_around_init, abins_inds_around_init[-1]+1)
area_bins_around_init = abins[abins_inds_around_init]
mean_cell_length_around_init = mean_cell_lengths[abins_inds_around_init[0:-1]]
max_cell_length_around_init = np.max(mean_cell_length_around_init)
lbins_inds_around_init = np.where(np.logical_and(-0.5*max_cell_length_around_init <= lbins , lbins <= 0.5*max_cell_length_around_init))
lbins_inds_around_init = lbins_inds_around_init[0]
lbins_inds_around_init = np.insert(lbins_inds_around_init, 0, [lbins_inds_around_init[0]-2, lbins_inds_around_init[0]-1])
lbins_inds_around_init = np.append(lbins_inds_around_init, [lbins_inds_around_init[-1]+1, lbins_inds_around_init[-1]+2])
lbins_around_init = lbins[lbins_inds_around_init]

heatmap_around_init = heatmap[np.ix_(abins_inds_around_init, lbins_inds_around_init)]

plot_abins_init = (area_bins_around_init[0:-1] + area_bins_around_init[1:])/2
plt.matshow(heatmap_around_init, aspect='auto', interpolation='none', 
            extent=[lbins_around_init[0], lbins_around_init[-1], area_bins_around_init[-1], area_bins_around_init[0]], origin='upper', cmap='jet')
plt.plot(-0.5 * mean_cell_length_around_init, y[abins_inds_around_init[0:-1]], 'w', linewidth=2)
plt.plot(0.5 * mean_cell_length_around_init, y[abins_inds_around_init[0:-1]], 'w', linewidth=2)
plt.axhline(init_area, color='red', linestyle='--', linewidth=2)
#[plt.axhline(y, color='magenta', linestyle='--') for y in y_centers]
plt.xlabel('Cell long axis (µm)')
plt.ylabel('Cell size (µm$^2$)')
plt.ylim(1.7, 1.35)



(1.7, 1.35)

In [16]:
amin_init = np.min(area_bins_around_init)
amax_init = np.max(area_bins_around_init)
lmin_init = np.min(lbins_around_init)
lmax_init = np.max(lbins_around_init)
ar_pix = areas * pixel_size**2
le_pix = lengths * pixel_size
lo_cent = longs - 0.5
len_long_rescaled = le_pix * lo_cent
inds_around_init = np.where(np.logical_and(np.logical_and(ar_pix >= amin_init, ar_pix <= amax_init), 
                                                   np.logical_and(len_long_rescaled >= lmin_init, 
                                                   len_long_rescaled <= lmax_init)))[0]
lo_init = lo_cent[inds_around_init]
nr_dots = lo_init.size

In [24]:

all_pos_numbers = all_data['position'].to_numpy()
pos_inds = np.unique(all_pos_numbers)
all_trap_numbers = all_data['trap'].to_numpy()
trap_nrs = np.unique(all_trap_numbers)
trap_nrs= trap_nrs[~np.isnan(trap_nrs)]

heatmap_around_init_dim = np.shape(heatmap_around_init)
heatmap_around_init_traps = np.zeros(shape=(heatmap_around_init_dim[0], heatmap_around_init_dim[-1], len(pos_inds)*len(trap_nrs)))
heatmap_around_init_pos_traps = np.zeros(shape=(heatmap_around_init_dim[0], heatmap_around_init_dim[-1], len(pos_inds), len(trap_nrs)))
all_dots_per_trap = np.zeros(shape=(len(pos_inds)*len(trap_nrs)))

k = 0
for i in range(0, len(pos_inds)):
    for j in range(0, len(trap_nrs)):
        #print(f'Position {pos_inds[i]} Trap {trap_nrs[j]+1}')  
        trap_inds = np.where((all_pos_numbers == pos_inds[i]) & (all_trap_numbers == trap_nrs[j]))
        areas_trap = areas[trap_inds]
        longs_trap = longs[trap_inds]
        lengths_trap = lengths[trap_inds]
        counts_trap = counts[trap_inds]

        heatmap_trap, _, _, _, _ = generate_fork_plot(areas_trap, lengths_trap, longs_trap, counts_trap, abins=abins, lbins=lbins, 
                                                      bin_scale=bin_scale, pixel_size=pixel_size, heatmap_threshold=heatmap_threshold)


        ar_trap_pix = areas_trap * pixel_size**2
        le_trap_pix = lengths_trap * pixel_size
        lo_trap_cent = longs_trap - 0.5
        len_long_trap_rescaled = le_trap_pix * lo_trap_cent
        inds_trap_around_init = np.where((ar_trap_pix >= amin_init) & (ar_trap_pix <= amax_init) & 
                                         (len_long_trap_rescaled >= lmin_init) & (len_long_trap_rescaled <= lmax_init))[0]
        longs_init_trap = lo_trap_cent[inds_trap_around_init]
        nr_dots_trap = longs_init_trap.size
        all_dots_per_trap[k] = nr_dots_trap
        #print(nr_dots_trap)
        
        heatmap_trap_init = heatmap_trap[np.ix_(abins_inds_around_init, lbins_inds_around_init)]
        heatmap_around_init_traps[:,:,k] = heatmap_trap_init
        heatmap_around_init_pos_traps[:,:,i,j] = heatmap_trap_init
        k += 1
        





In [25]:
np.savez(save_dir / Path('heatmaps_per_trap_around_init.npz'), var1=heatmap_around_init_traps, var2=heatmap_around_init_pos_traps, var3=all_dots_per_trap)



In [26]:
loaded_data = (np.load(save_dir / Path('heatmaps_per_trap_around_init.npz')))

heatmap_around_init_traps = loaded_data['var1']
heatmap_around_init_pos_traps = loaded_data['var2']
tot_nr_traps = heatmap_around_init_traps.shape[-1]

flat_heatmap_around_init = heatmap_around_init.flatten()

weight = pysal.lib.weights.lat2W(heatmap_around_init.shape[0], heatmap_around_init.shape[-1], rook=True) 

moran_flat_heatmap = esda.moran.Moran(flat_heatmap_around_init, weight)

#Calculations for the energy test
x_centers = np.linspace(lbins_around_init[0], lbins_around_init[-1], num=36)
y_centers = np.linspace(area_bins_around_init[-1], area_bins_around_init[0], num=8)
xx, yy = np.meshgrid(x_centers, y_centers)
bin_centers = np.column_stack((xx.ravel(), yy.ravel()))
dists = squareform(pdist(bin_centers))


#### All traps, no position information scoring

In [27]:
corrcoeffs = np.zeros(shape=(tot_nr_traps, ))
all_ssims = np.zeros(shape=(tot_nr_traps, ))
all_moran_bivars = np.zeros(shape=(tot_nr_traps, 3))
ks_results = np.zeros(shape=(tot_nr_traps, 2))
#chi_results = np.zeros(shape=(tot_nr_traps,2))
sobolev_results = np.zeros(shape=(tot_nr_traps, ))
energy_test_results = np.zeros(shape=(tot_nr_traps, ))

for i in range(0, tot_nr_traps):
    flat_trap_heatmap = heatmap_around_init_traps[:,:,i].flatten()
    
    #Pearson correlation coefficient 
    corrcoeffs[i] = np.corrcoef(flat_heatmap_around_init, flat_trap_heatmap)[0, 1]

    #SSIM
    max_val = np.max([np.max(flat_heatmap_around_init), np.max(flat_trap_heatmap)])
    min_val = np.min([np.min(flat_heatmap_around_init), np.min(flat_trap_heatmap)])
    all_ssims[i] = ssim(flat_heatmap_around_init, flat_trap_heatmap, data_range=max_val-min_val)

    #Moran's I
    #moran_flat_heatmap_trap = esda.moran.Moran(flat_trap_heatmap, weight)
    #print(f"Moran's I: {moran_flat_heatmap_trap.I}, p-value: {moran_flat_heatmap_trap.p_norm}")
    moran_bivar_heatmap = esda.moran.Moran_BV(flat_heatmap_around_init, flat_trap_heatmap, weight)
    all_moran_bivars[i,0] = moran_bivar_heatmap.I
    all_moran_bivars[i,1] = moran_bivar_heatmap.p_sim
    all_moran_bivars[i,2] = moran_bivar_heatmap.z_sim

    #Kolmogorov-Smirnov
    ks_stat, ks_pval = stats.ks_2samp(flat_heatmap_around_init, flat_trap_heatmap)
    ks_results[i,0] = ks_stat
    ks_results[i,1] = ks_pval

    #chi_stats, chi_pval = stats.chisquare(flat_trap_heatmap, flat_heatmap_around_init)
    #chi_results[:,0] = chi_stats
    #chi_results[:,1]= chi_pval
    
    #Sobolev
    diff_heatmap = heatmap_around_init - heatmap_around_init_traps[:,:,i]
    fft_diff_heatmap = fft2(diff_heatmap)
    ny, nx = diff_heatmap.shape
    kx = fftfreq(nx)
    ky = fftfreq(ny)

    kx_2d, ky_2d = np.meshgrid(kx, ky)
    k_squared = kx_2d**2 + ky_2d**2
    k_squared[0,0] = 1

    weighted_sum_heatmaps = np.sum(np.abs(fft_diff_heatmap)**2 / (1+4*np.pi**2*k_squared))
    sobolev_norm = np.sqrt(weighted_sum_heatmaps)
    sobolev_results[i] = sobolev_norm

    #Energy test
    n1, n2 = np.sum(flat_heatmap_around_init), np.sum(flat_trap_heatmap)
    w1, w2 = flat_heatmap_around_init/n1, flat_trap_heatmap/n2

    phi_11 = np.sum(np.outer(w1, w1) * dists)
    phi_22 = np.sum(np.outer(w2, w2) * dists)
    phi_12 = np.sum(np.outer(w1, w2) * dists)
    energy = phi_11 + phi_22 - 2*phi_12

    energy_test_results[i] = energy
    
    #print(f"Cross-Moran's I between full fork plot and per trap fork plot (trap: {i+1}): {moran_bivar_heatmap.I}, p-value: {moran_bivar_heatmap.p_sim}, 'z-score: {moran_bivar_heatmap.z_sim}")
    

In [28]:
med_corrcoeff = np.nanmedian(corrcoeffs)
med_ssim = np.nanmedian(all_ssims)
med_moran = np.nanmedian(all_moran_bivars[:,0])
med_ks = np.nanmedian(ks_results[:,0])
med_sobolev = np.nanmedian(sobolev_results)
med_energy = np.nanmedian(energy_test_results)

mad_corrcoeff = stats.median_abs_deviation(corrcoeffs, nan_policy='omit')
mad_ssim = stats.median_abs_deviation(all_ssims, nan_policy='omit')
mad_moran = stats.median_abs_deviation(all_moran_bivars[:,0], nan_policy='omit')
mad_ks = stats.median_abs_deviation(ks_results[:,0], nan_policy='omit')
mad_sobolev = stats.median_abs_deviation(sobolev_results, nan_policy='omit')
mad_energy = stats.median_abs_deviation(energy_test_results, nan_policy='omit')

med_mad_up_corrcoeff = med_corrcoeff+mad_corrcoeff
med_mad_down_corrcoeff = med_corrcoeff-mad_corrcoeff
med_mad_up_ssim = med_ssim+mad_ssim
med_mad_down_ssim = med_ssim-mad_ssim
med_mad_up_moran = med_moran+mad_moran
med_mad_down_moran = med_moran-mad_moran
med_mad_up_ks = med_ks+mad_ks
med_mad_down_ks = med_ks-mad_ks
med_mad_up_sobolev = med_sobolev+mad_sobolev
med_mad_down_sobolev = med_sobolev-mad_sobolev
med_mad_up_energy = med_energy+mad_energy
med_mad_down_energy = med_energy-mad_energy



In [30]:
plt.close('all')

plot_range = np.arange(1, tot_nr_traps+1, 1)
x_fill = np.array([1, tot_nr_traps, tot_nr_traps, 1])

plt.figure(1)
plt.plot(plot_range, corrcoeffs, 'o')
plt.axhline(med_corrcoeff, linestyle='--', color='red')
plt.fill(x_fill, np.array([med_mad_down_corrcoeff, med_mad_down_corrcoeff, med_mad_up_corrcoeff, med_mad_up_corrcoeff]), 
         color='red', alpha=0.25, zorder=3) 
plt.xlabel('Channel')
plt.ylabel('Pearson correlation coefficient') 

plt.figure(2)
plt.plot(plot_range, all_ssims, 'o')
plt.axhline(med_ssim, linestyle='--', color='red')
plt.fill(x_fill, np.array([med_mad_down_ssim, med_mad_down_ssim, med_mad_up_ssim, med_mad_up_ssim]), color='red', alpha=0.25, zorder=3) 
plt.xlabel('Channel')
plt.ylabel('SSIM') 

plt.figure(3)
plt.plot(plot_range, all_moran_bivars[:,0], 'o')
plt.axhline(med_moran, linestyle='--', color='red')
plt.fill(x_fill, np.array([med_mad_down_moran, med_mad_down_moran, med_mad_up_moran, med_mad_up_moran]), color='red', alpha=0.25, zorder=3) 
plt.xlabel('Channel')
plt.ylabel('Cross-Moran\'s I')

plt.figure(4)
plt.plot(plot_range, all_moran_bivars[:,1], 'o')
plt.xlabel('Channel')
plt.ylabel('Cross-Moran\'s I, p-value')

plt.figure(5)
plt.plot(plot_range, all_moran_bivars[:,2], 'o')
plt.xlabel('Channel')
plt.ylabel('Cross-Moran\'s I, z-score')

plt.figure(6)
plt.plot(plot_range, ks_results[:,0], 'o')
plt.axhline(med_ks, linestyle='--', color='red')
plt.fill(x_fill, np.array([med_mad_down_ks, med_mad_down_ks, med_mad_up_ks, med_mad_up_ks]), color='red', alpha=0.25, zorder=3) 
plt.xlabel('Channel')
plt.ylabel('KS statistic')

plt.figure(7)
plt.plot(plot_range, ks_results[:,1], 'o')
plt.xlabel('Channel')
plt.ylabel('KS statistic p-value')

plt.figure(8)
plt.plot(plot_range, sobolev_results, 'o')
plt.axhline(med_sobolev, linestyle='--', color='red')
plt.fill(x_fill, np.array([med_mad_down_sobolev, med_mad_down_sobolev, med_mad_up_sobolev, med_mad_up_sobolev]), color='red', alpha=0.25, zorder=3) 
plt.xlabel('Channel')
plt.ylabel('Sobolev norm')

plt.figure(9)
plt.plot(plot_range, energy_test_results, 'o')
plt.axhline(med_energy, linestyle='--', color='red')
plt.fill(x_fill, np.array([med_mad_down_energy, med_mad_down_energy, med_mad_up_energy, med_mad_up_energy]), color='red', alpha=0.25, zorder=3)  
plt.xlabel('Channel')
plt.ylabel('Energy test result')

Text(0, 0.5, 'Energy test result')

In [32]:
plt.close('all')

plt.figure()
plt.plot(all_dots_per_trap, corrcoeffs, 'o')
plt.xlabel('Number dots around initiation')
plt.ylabel('Pearson correlation coefficient')

plt.figure()
plt.plot(all_dots_per_trap, all_ssims, 'o')
plt.xlabel('Number dots around initiation')
plt.ylabel('SSIM')

plt.figure()
plt.plot(all_dots_per_trap, all_moran_bivars[:,0], 'o')
plt.xlabel('Number dots around initiation')
plt.ylabel('Cross Moran\'s I')

plt.figure()
plt.plot(all_dots_per_trap, ks_results[:,0], 'o')
plt.xlabel('Number dots around initiation')
plt.ylabel('KS statistic')

plt.figure()
plt.plot(all_dots_per_trap, sobolev_results, 'o')
plt.xlabel('Number dots around initiation')
plt.ylabel('Sobolev norm')

plt.figure()
plt.plot(all_dots_per_trap, energy_test_results, 'o')
plt.xlabel('Number dots around initiation')
plt.ylabel('Energy test')


Text(0, 0.5, 'Energy test')

#### Position information scoring

In [17]:
all_pos_numbers = all_data['position'].to_numpy()
pos_inds = np.unique(all_pos_numbers)
nr_pos = len(pos_inds)
all_trap_numbers = all_data['trap'].to_numpy()
trap_nrs = np.unique(all_trap_numbers)
trap_nrs= trap_nrs[~np.isnan(trap_nrs)]
nr_traps = len(trap_nrs)
corrcoeffs_pos_trap = np.zeros(shape=(nr_pos, nr_traps))
all_ssims_pos_trap = np.zeros(shape=(nr_pos, nr_traps))
all_moran_bivars_pos_trap = np.zeros(shape=(nr_pos, nr_traps, 3))
ks_results_pos_trap = np.zeros(shape=(nr_pos, nr_traps, 2))
#chi_results = np.zeros(shape=(tot_nr_traps,2))
sobolev_results_pos_trap = np.zeros(shape=(nr_pos, nr_traps))
energy_test_results_pos_trap = np.zeros(shape=(nr_pos, nr_traps))

for i in range(0, len(pos_inds)):
    for j in range(0, len(trap_nrs)):
        flat_pos_trap_heatmap = heatmap_around_init_pos_traps[:,:,i,j].flatten()

        #Correlation coefficient 
        corrcoeffs_pos_trap[i, j] = np.corrcoef(flat_heatmap_around_init, flat_pos_trap_heatmap)[0, 1]

        #SSIM
        max_val = np.max([np.max(flat_heatmap_around_init), np.max(flat_pos_trap_heatmap)])
        min_val = np.min([np.min(flat_heatmap_around_init), np.min(flat_pos_trap_heatmap)])
        all_ssims_pos_trap[i,j] = ssim(flat_heatmap_around_init, flat_pos_trap_heatmap, data_range=max_val-min_val)
        
        #Moran's I
        moran_bivar_heatmap_pos_trap = esda.moran.Moran_BV(flat_heatmap_around_init, flat_pos_trap_heatmap, weight)
        all_moran_bivars_pos_trap[i,j,0] = moran_bivar_heatmap_pos_trap.I
        all_moran_bivars_pos_trap[i,j,1] = moran_bivar_heatmap_pos_trap.p_sim
        all_moran_bivars_pos_trap[i,j,2] = moran_bivar_heatmap_pos_trap.z_sim

        #Kolmogorov-Smirnov
        ks_stat_pos_trap, ks_pval_pos_trap = stats.ks_2samp(flat_heatmap_around_init, flat_pos_trap_heatmap)
        ks_results_pos_trap[i,j,0] = ks_stat_pos_trap
        ks_results_pos_trap[i,j,1] = ks_pval_pos_trap

        #Sobolev
        diff_heatmap = heatmap_around_init - heatmap_around_init_pos_traps[:,:,i,j]
        fft_diff_heatmap = fft2(diff_heatmap)
        ny, nx = diff_heatmap.shape
        kx = fftfreq(nx)
        ky = fftfreq(ny)

        kx_2d, ky_2d = np.meshgrid(kx, ky)
        k_squared = kx_2d**2 + ky_2d**2
        k_squared[0,0] = 1

        weighted_sum_heatmaps = np.sum(np.abs(fft_diff_heatmap)**2 / (1+4*np.pi**2*k_squared))
        sobolev_norm = np.sqrt(weighted_sum_heatmaps)
        sobolev_results_pos_trap[i,j] = sobolev_norm

        #Energy test
        n1, n2 = np.sum(flat_heatmap_around_init), np.sum(flat_pos_trap_heatmap)
        w1, w2 = flat_heatmap_around_init/n1, flat_pos_trap_heatmap/n2

        phi_11 = np.sum(np.outer(w1, w1) * dists)
        phi_22 = np.sum(np.outer(w2, w2) * dists)
        phi_12 = np.sum(np.outer(w1, w2) * dists)
        energy = phi_11 + phi_22 - 2*phi_12

        energy_test_results_pos_trap[i,j] = energy

In [21]:
plt.close('all')

plt.figure(11)
plt.imshow(corrcoeffs_pos_trap, cmap='viridis', aspect='auto')
plt.colorbar(label='Pearson correlation coefficient')
plt.xlabel('Trap')
plt.ylabel('Position')

plt.figure(12)
plt.imshow(all_ssims_pos_trap, cmap='viridis', aspect='auto')
plt.colorbar(label='SSIM')
plt.xlabel('Trap')
plt.ylabel('Position')

plt.figure(13)
plt.imshow(all_moran_bivars_pos_trap[:,:,0], cmap='viridis', aspect='auto')
plt.colorbar(label='Cross-Moran\'s I')
plt.xlabel('Trap')
plt.ylabel('Position')

plt.figure(14)
plt.imshow(all_moran_bivars_pos_trap[:,:,1], cmap='viridis', aspect='auto')
plt.colorbar(label='Cross-Moran\'s I, p-value')
plt.xlabel('Trap')
plt.ylabel('Position')

plt.figure(15)
plt.imshow(all_moran_bivars_pos_trap[:,:,2], cmap='viridis', aspect='auto')
plt.colorbar(label='Cross-Moran\'s I, z-score')
plt.xlabel('Trap')
plt.ylabel('Position')

plt.figure(16)
plt.imshow(ks_results_pos_trap[:,:,0], cmap='viridis_r', aspect='auto')
plt.colorbar(label='KS statistic')
plt.xlabel('Trap')
plt.ylabel('Position')

plt.figure(17)
plt.imshow(ks_results_pos_trap[:,:,1], cmap='viridis', aspect='auto')
plt.colorbar(label='KS statistic, p-value')
plt.xlabel('Trap')
plt.ylabel('Position')

plt.figure(18)
plt.imshow(sobolev_results_pos_trap, cmap='viridis', aspect='auto')
plt.colorbar(label='Sobolev norm')
plt.xlabel('Trap')
plt.ylabel('Position')

plt.figure(19)
plt.imshow(energy_test_results_pos_trap, cmap='viridis', aspect='auto')
plt.colorbar(label='Energy test result')
plt.xlabel('Trap')
plt.ylabel('Position')


Text(0, 0.5, 'Position')

#### Checking if metrics are similar 

In [20]:
plt.close('all')

plt.figure()
plt.plot(corrcoeffs, all_ssims, 'o')
plt.xlabel('Pearson correlation coefficient')
plt.ylabel('SSIM')

plt.figure()
plt.plot(corrcoeffs, all_moran_bivars[:,0], 'o')
plt.xlabel('Pearson correlation coefficient')
plt.ylabel('Cross-Moran\'s I')

plt.figure()
plt.plot(corrcoeffs, ks_results[:,0], 'o')
plt.xlabel('Pearson correlation coefficient')
plt.ylabel('KS statistics')

plt.figure()
plt.plot(corrcoeffs, sobolev_results, 'o')
plt.xlabel('Pearson correlation coefficient')
plt.ylabel('Sobolev norm')

plt.figure()
plt.plot(corrcoeffs, energy_test_results, 'o')
plt.xlabel('Pearson correlation coefficient')
plt.ylabel('Energy test results')

plt.figure()
plt.plot(all_ssims, all_moran_bivars[:,0], 'o')
plt.xlabel('SSIM')
plt.ylabel('Cross-Moran\'s I')

plt.figure()
plt.plot(all_ssims, ks_results[:,0], 'o')
plt.xlabel('SSIM')
plt.ylabel('KS statistics')

plt.figure()
plt.plot(all_ssims, sobolev_results, 'o')
plt.xlabel('SSIM')
plt.ylabel('Sobolev norm')

plt.figure()
plt.plot(all_ssims, energy_test_results, 'o')
plt.xlabel('SSIM')
plt.ylabel('Energy test results')

plt.figure()
plt.plot(all_moran_bivars[:,0], ks_results[:,0], 'o')
plt.xlabel('Cross-Moran\'s I')
plt.ylabel('KS statistics')

plt.figure()
plt.plot(all_moran_bivars[:,0], sobolev_results, 'o')
plt.xlabel('Cross-Moran\'s I')
plt.ylabel('Sobolev norm')
           
plt.figure()
plt.plot(all_moran_bivars[:,0], energy_test_results, 'o')
plt.xlabel('Cross-Moran\'s I')
plt.ylabel('Energy test results')

plt.figure()
plt.plot(ks_results[:,0], sobolev_results, 'o')
plt.xlabel('KS statistics')
plt.ylabel('Sobolev norm')

plt.figure()
plt.plot(ks_results[:,0], energy_test_results, 'o')
plt.xlabel('KS statistics')
plt.ylabel('Energy test results')

plt.figure()
plt.plot(sobolev_results, energy_test_results, 'o')
plt.xlabel('Sobolev norm')
plt.ylabel('Energy test results')

Text(0, 0.5, 'Energy test results')

In [31]:
x = [0, 10, 10, 0]
y = [0, 0, 2, 2]

# Create the plot
plt.figure(figsize=(8, 6))
plt.fill(x, y, alpha=0.5, color='blue')

In [ ]:
def precompute_forks(self, clicked):
    sys.stdout.write("Pre-computing forks for scoring\n")
    sys.stdout.flush()

    # Check if data has already been computed
    file_dir_hdf5 = Path(self.param.Save.directory) / Path('fork_data.h5')
    file_dir_moran = Path(self.param.Save.directory) / Path('moran_weights.gal')
    
    if exists(file_dir_hdf5) and exists(file_dir_moran):
        with h5py.File(file_dir_hdf5, 'r') as hdf:
            self.fork_data = {}
            for key in hdf.keys():
                self.fork_data[key] = hdf[key][()]
        
        self.moran_weight = libpysal.io.open(file_dir_moran).read()
        sys.stdout.write("Loaded fork data from file\n")
        sys.stdout.flush()
    else:
        self.all_fork_fetch_thread = PreComputeForkFetchThread(self.param)
        self.all_fork_fetch_thread.fork_fetched.connect(self.precomputed_fork_data)
        self.all_fork_fetch_thread.start()

def precomputed_fork_data(self):
    file_dir_hdf5 = Path(self.param.Save.directory) / Path('fork_data.h5')
    file_dir_moran = Path(self.param.Save.directory) / Path('moran_weights.gal')
    
    with h5py.File(file_dir_hdf5, 'r') as hdf:
        self.fork_data = {}
        for key in hdf.keys():
            self.fork_data[key] = hdf[key][()]
    
    self.moran_weight = libpysal.io.open(file_dir_moran).read()
    sys.stdout.write("Loaded precomputed fork data from file\n")
    sys.stdout.flush()


In [ ]:
def precompute_forks(self, clicked):
    sys.stdout.write("Pre-computing forks for scoring\n")
    sys.stdout.flush()

    # Check if data has already been computed
    file_dir_hdf5 = Path(self.param.Save.directory) / Path('fork_data.h5')
    file_dir_moran = Path(self.param.Save.directory) / Path('moran_weights.gal')
    
    if ~exists(file_dir_hdf5) and ~exists(file_dir_moran):
        self.all_fork_fetch_thread = PreComputeForkFetchThread(self.param)
        self.all_fork_fetch_thread.fork_fetched.connect(self.precomputed_fork_data)
        self.all_fork_fetch_thread.start()

def precomputed_fork_data(self):
    file_dir_hdf5 = Path(self.param.Save.directory) / Path('fork_data.h5')
    file_dir_moran = Path(self.param.Save.directory) / Path('moran_weights.gal')
    
    with h5py.File(file_dir_hdf5, 'r') as hdf:
        self.fork_data = {}
        for key in hdf.keys():
            self.fork_data[key] = hdf[key][()]
    
    self.moran_weight = libpysal.io.open(file_dir_moran).read()
    sys.stdout.write("Loaded precomputed fork data from file\n")
    sys.stdout.flush()


In [ ]:
 def precompute_forks(self, clicked):
        sys.stdout.write("Pre-computing forks for scoring\n")
        sys.stdout.flush()


        if self.precomputed_fork_thread is None:
            self.precomputed_fork_thread = PreComputeForkFetchThread(self.param)

        #Do a check if data has already been computed, maybe if a file exists?
            file_dir_hdf5 = Path(self.param.Save.directory) / Path('fork_data.h5')
            file_dir_moran = Path(self.param.Save.directory) / Path('moran_weights.gal')
        
            if not exists(file_dir_hdf5) and not exists(file_dir_moran):
                self.precomputed_fork_thread.start()
        
            print(self.precomputed_fork_thread)
            self.precomputed_fork_thread.fork_fetched.connect(self.precomputed_fork_bdata)

 def precomputed_fork_data(self):
        
        fork_data, moran_weights = self.precomputed_fork_thread.get_data()

        print(fork_data, moran_weights)

In [ ]:
def precompute_forks(self, clicked):
    sys.stdout.write("Pre-computing forks for scoring\n")
    sys.stdout.flush()

    if self.precomputed_fork_thread is None:
        self.precomputed_fork_thread = PreComputeForkFetchThread(self.param)

    file_dir_hdf5 = Path(self.param.Save.directory) / Path('fork_data.h5')
    file_dir_moran = Path(self.param.Save.directory) / Path('moran_weights.gal')

    if not exists(file_dir_hdf5) and not exists(file_dir_moran):
        self.precomputed_fork_thread.start()
    else:
        # If files exist, manually call the precomputed_fork_data method
        # or emit the fork_fetched signal to trigger it
        self.precomputed_fork_thread.fork_fetched.emit()

    # Connect the signal here, regardless of whether files exist
        self.precomputed_fork_thread.fork_fetched.connect(self.precomputed_fork_data)


In [11]:
test = np.zeros(shape=(2,2))
print(test)

[[0. 0.]
 [0. 0.]]


In [12]:
print(test[:,:,0])

IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed